In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from src.load import load_full_network_from_df
from src.util import FullTransferNetwork

from collections import defaultdict, namedtuple

import math

In [2]:
AggregateFullTransferNetwork = namedtuple(
    'AggregateFullTransferNetwork',
    ['G', 'league_clubs', 'currency', 'denomination', 'player_key', 'fee_key', 'year_key', 'window_key', 'start_year', 'end_year']
)
def load_full_aggregate_network(
    start_year: int = 2000, 
    end_year: int = 2010, 
    league: str = "english_premier_league"
) -> AggregateFullTransferNetwork:
    '''
    Create an aggregate Full Transfer Network from start year to end year. The graph will contain 
    attributes such as players, teams, fees, years, and transfer windows
    
    Inputs:
        start_year:  (int) the start year to look at for transfers. Lowest value 1888
        end_year:    (int) the end year to look at for transfers. Highest value is 2019
        league:      (str) the league to load
    Outputs:
        AggregateFullTransferNetwork
    '''
    
    # create the graph to use
    G = nx.MultiDiGraph()
    
    # keep track of all clubs
    clubs = []
    
    # keep track of the keys
    player_key = None
    fee_key = None
    year_key = None
    window_key = None
    denomination = None
    
    # go through all years and load the networks
    for year in range(start_year, end_year + 1):
        
        # we have to make the dataframe first
        df = pd.read_csv(f"data/{year}/{league}.csv")
        
        year_ftn = load_full_network_from_df(df, "pounds")
        
        # set the keys
        if year == start_year:
            player_key = year_ftn.player_key
            fee_key = year_ftn.fee_key
            year_key = year_ftn.year_key
            window_key = year_ftn.window_key
            denomination = year_ftn.denomination
        
        # get all clubs
        clubs += year_ftn.league_clubs
        
        # combine the graphs
        for u, v, data in year_ftn.G.edges(data=True):
            G.add_edge(u, v, player=data[player_key], fee=data[fee_key], year=data[year_key], transfer_window=data[window_key])
        
    # setify the clubs
    clubs = list(set(clubs))
    
    # now create the AggregateFullTransferNetwork
    return AggregateFullTransferNetwork(G, clubs, "pounds", denomination, player_key, fee_key, year_key, window_key, start_year, end_year)




In [3]:
aftn = load_full_aggregate_network()

In [4]:
for u, v, data in aftn.G.edges(data=True):
    if 'dimitar berbatov' in data[aftn.player_key]:
        print(f'from {u} to {v} with data {data}')

from tottenham to manchester united with data {'player': 'dimitar berbatov', 'fee': 34.2, 'year': 2008, 'transfer_window': 'summer'}
from bay. leverkusen to tottenham with data {'player': 'dimitar berbatov', 'fee': 14.13, 'year': 2006, 'transfer_window': 'summer'}
